# 1.기본 라이브러리 import

In [1]:
import pandas as pd
import numpy as np
import folium
from folium import plugins
import warnings
import matplotlib.pyplot as plt
import json
from sklearn.metrics import accuracy_score, classification_report
import sklearn.neighbors as neg
import sklearn.preprocessing as pp
from sklearn.cluster import KMeans
import seaborn as sns
from sklearn.metrics import silhouette_samples
from matplotlib import cm
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from scipy.spatial.distance import cdist, pdist
from sklearn.preprocessing import MinMaxScaler
from haversine import haversine
from tqdm.notebook import tqdm
%matplotlib inline
import platform
import pickle
import re
platform.sys
warnings.filterwarnings(action='ignore')
plt.rc('font',family='Malgun Gothic')

# 2.데이터 불러오기 및 전처리
사용한 데이터 목록
우리동네키움센터현황(2020, 우리동네키움센터홈페이지)
청소년유해업소현황(2020,서울특별시청평생교육국)
공시지가(2020,국가공간정보포털)
국민연금공단_자격 시구신고(2020.11, 공공데이터포털)
방과후_활동_617세(2019, 공공데이터포털)
서울시 행정구역 시군구 정보 (좌표계: ITRF2000)(2020, 서울정보소통광장)
서울시_행정동_법정동(2020,서울 열린데이터 광장)
초등돌봄교실이용현황_서울(2020,교육통계시스템)
초등학생_자녀의_현재_돌봄방법(2020,kosis)
행정_법정동 중심좌표(2020,국가공간정보퍼털)
귀가_시__부모님_등_어른의_맞이_횟수(2018,국가정보포털)
연령별_성별_혼인상태별_인구_15세이상_내국인_시군구(kosis,2019) 초등학교 현황(2020, 교육통계시스템)
초등학생_자녀의_현재_돌봄방법(2020,kosis)
서울시_행정동_법정동(2020,kosis)
지역아동센터 (웹크롤링)
행정동경계정보(2020,통계청)
우리동네키움센터_현황 (기관 문의)

# 일부 전처리 생략



In [2]:
standard=pd.read_csv('data/json_dong.csv',encoding="CP949")

In [2]:
# 메인으로 삼을 동(법정동, 행정동 주소가 차이가 존재.)

In [3]:
standard

,dong
0,길동
1,천호2동
2,암사1동
3,둔촌2동
4,둔촌1동
...,...
418,무악동
419,평창동
420,부암동
421,삼청동


In [4]:
kid=pd.read_csv('data/0_11_Seoul_v4.csv',encoding = 'CP949')
kid.head()

,행정구,행정동,법정코드,2020년10월_계_총인구수,2020년10월_계_연령구간인구수,2020년10월_계_0세,2020년10월_계_1세,2020년10월_계_2세,2020년10월_계_3세,2020년10월_계_4세,2020년10월_계_5세,2020년10월_계_6세,2020년10월_계_7세,2020년10월_계_8세,2020년10월_계_9세,2020년10월_계_10세,2020년10월_계_11세
0,서울특별시,NaN,1.100000e+09,9689159,774577,45545,51348,54128,58242,66101,68808,68673,69474,75219,73920,72653,70466
1,종로구,전체,1.111000e+09,149703,9581,516,592,581,689,793,840,880,849,976,945,958,962
2,종로구,청운효자동,1.111052e+09,12446,1063,53,65,57,90,74,102,93,89,121,97,105,117
3,종로구,사직동,1.111053e+09,9538,704,25,40,42,52,63,70,77,72,68,67,63,65
4,종로구,삼청동,1.111054e+09,2673,136,8,4,8,12,5,13,13,10,12,18,10,23


In [5]:
kid.dropna(inplace=True)

In [6]:
kid

,행정구,행정동,법정코드,2020년10월_계_총인구수,2020년10월_계_연령구간인구수,2020년10월_계_0세,2020년10월_계_1세,2020년10월_계_2세,2020년10월_계_3세,2020년10월_계_4세,2020년10월_계_5세,2020년10월_계_6세,2020년10월_계_7세,2020년10월_계_8세,2020년10월_계_9세,2020년10월_계_10세,2020년10월_계_11세
1,종로구,전체,1.111000e+09,149703,9581,516,592,581,689,793,840,880,849,976,945,958,962
2,종로구,청운효자동,1.111052e+09,12446,1063,53,65,57,90,74,102,93,89,121,97,105,117
3,종로구,사직동,1.111053e+09,9538,704,25,40,42,52,63,70,77,72,68,67,63,65
4,종로구,삼청동,1.111054e+09,2673,136,8,4,8,12,5,13,13,10,12,18,10,23
5,종로구,부암동,1.111055e+09,10018,722,29,44,31,53,59,64,54,84,69,70,82,83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,강동구,성내제3동,1.174066e+09,23448,1602,130,120,140,147,139,137,125,143,105,134,133,149
542,강동구,길동,1.174068e+09,46641,3151,192,242,226,252,281,287,307,246,294,262,308,254
543,강동구,둔촌제1동,1.174069e+09,258,9,0,0,0,0,0,2,0,2,1,1,1,2
544,강동구,둔촌제2동,1.174070e+09,26849,2250,146,155,156,171,193,202,216,195,237,200,198,181


In [7]:
kid.rename(columns={"행정구":"gu","행정동":"dong","2020년10월_계_연령구간인구수":"total"},inplace=True)

In [8]:
kid.head()

,gu,dong,법정코드,2020년10월_계_총인구수,total,2020년10월_계_0세,2020년10월_계_1세,2020년10월_계_2세,2020년10월_계_3세,2020년10월_계_4세,2020년10월_계_5세,2020년10월_계_6세,2020년10월_계_7세,2020년10월_계_8세,2020년10월_계_9세,2020년10월_계_10세,2020년10월_계_11세
1,종로구,전체,1.111000e+09,149703,9581,516,592,581,689,793,840,880,849,976,945,958,962
2,종로구,청운효자동,1.111052e+09,12446,1063,53,65,57,90,74,102,93,89,121,97,105,117
3,종로구,사직동,1.111053e+09,9538,704,25,40,42,52,63,70,77,72,68,67,63,65
4,종로구,삼청동,1.111054e+09,2673,136,8,4,8,12,5,13,13,10,12,18,10,23
5,종로구,부암동,1.111055e+09,10018,722,29,44,31,53,59,64,54,84,69,70,82,83


In [9]:
kid.drop(['법정코드', '2020년10월_계_총인구수',
       '2020년10월_계_0세', '2020년10월_계_1세', '2020년10월_계_2세', '2020년10월_계_3세',
       '2020년10월_계_4세', '2020년10월_계_5세', '2020년10월_계_6세', '2020년10월_계_7세',
       '2020년10월_계_8세', '2020년10월_계_9세', '2020년10월_계_10세', '2020년10월_계_11세',],axis=1,inplace=True)

In [10]:
kid=kid.drop(kid[kid['dong']=='전체'].index)

In [11]:
kid

,gu,dong,total
2,종로구,청운효자동,1063
3,종로구,사직동,704
4,종로구,삼청동,136
5,종로구,부암동,722
6,종로구,평창동,1475
...,...,...,...
540,강동구,성내제3동,1602
542,강동구,길동,3151
543,강동구,둔촌제1동,9
544,강동구,둔촌제2동,2250


In [12]:
standard.head()

,dong
0,길동
1,천호2동
2,암사1동
3,둔촌2동
4,둔촌1동


In [13]:
kid.head()

,gu,dong,total
2,종로구,청운효자동,1063
3,종로구,사직동,704
4,종로구,삼청동,136
5,종로구,부암동,722
6,종로구,평창동,1475


In [14]:
kid=kid.reset_index()
kid.drop(['index'],axis=1,inplace=True)
kid.head()

,gu,dong,total
0,종로구,청운효자동,1063
1,종로구,사직동,704
2,종로구,삼청동,136
3,종로구,부암동,722
4,종로구,평창동,1475


In [15]:
gu_list=[]
for i in range(len(standard)):
    gu_list.append(0)
    for j in range(len(kid)):
        if standard.dong[i]==kid.dong[j]:
            gu_list[i]=kid.gu[j]
            break;
    

In [16]:
standard

,dong
0,길동
1,천호2동
2,암사1동
3,둔촌2동
4,둔촌1동
...,...
418,무악동
419,평창동
420,부암동
421,삼청동


In [17]:
gu_list

['강동구',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '강동구',
 '강동구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '송파구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '강남구',
 '은평구',
 '서초구',
 '서초구',
 '서초구',
 '서초구',
 '서초구',
 '서초구',
 '서초구',
 '서초구',
 '서초구',
 '서초구',
 '서초구',
 '서초구',
 '서초구',
 '서초구',
 '서초구',
 '서초구',
 '서초구',
 '서초구',
 '관악구',
 '관악구',
 '관악구',
 '관악구',
 '관악구',
 '관악구',
 '관악구',
 '관악구',
 '관악구',
 '관악구',
 '은평구',
 '관악구',
 '관악구',
 '관악구',
 '관악구',
 '관악구',
 '관악구',
 '관악구',
 '관악구',
 '관악구',
 '관악구',
 0,
 0,
 '동작구',
 0,
 0,
 '동작구',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '영등포구',
 '영등포구',
 '영등포구',
 '영등포구',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '금천구',
 0,
 0,
 '구로구',


In [18]:
standard['gu']=gu_list

In [19]:
standard['gu']

0      강동구
1        0
2        0
3        0
4        0
      ... 
418    종로구
419    종로구
420    종로구
421    종로구
422    종로구
Name: gu, Length: 423, dtype: object

In [20]:
standard

,dong,gu
0,길동,강동구
1,천호2동,0
2,암사1동,0
3,둔촌2동,0
4,둔촌1동,0
...,...,...
418,무악동,종로구
419,평창동,종로구
420,부암동,종로구
421,삼청동,종로구


In [21]:
standard.to_csv('standard.csv',encoding='euc-kr')

In [22]:
standard2=pd.read_csv('standard_2.csv',encoding="CP949")

In [23]:
standard2.drop(['Unnamed: 0'],axis=1,inplace=True)
standard2.head()

,dong,gu
0,길동,강동구
1,천호2동,강동구
2,암사1동,강동구
3,둔촌2동,강동구
4,둔촌1동,강동구


In [3]:
#동 주소를 기반으로 무슨 구인지, 타겟팅 나이대 인원 수 데이터를 합침

In [24]:
kid.head()

,gu,dong,total
0,종로구,청운효자동,1063
1,종로구,사직동,704
2,종로구,삼청동,136
3,종로구,부암동,722
4,종로구,평창동,1475


In [25]:
total_list=[]
for i in range(len(standard)):
    total_list.append(0)
    for j in range(len(kid)):
        if standard.dong[i]==kid.dong[j]:
            total_list[i]=kid.total[j]
            break;

In [26]:
total_list

[3151,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4991,
 3669,
 4208,
 5539,
 739,
 1849,
 3503,
 1622,
 2913,
 1971,
 1643,
 3288,
 3742,
 1960,
 2017,
 1982,
 2000,
 1515,
 3435,
 1906,
 1425,
 1395,
 1181,
 1343,
 1598,
 848,
 2433,
 864,
 2048,
 2904,
 2643,
 2776,
 5714,
 734,
 1554,
 965,
 2954,
 1702,
 487,
 3845,
 2274,
 4032,
 1273,
 1049,
 3008,
 3119,
 1053,
 1379,
 844,
 3409,
 2444,
 1789,
 5318,
 2127,
 1685,
 1494,
 1432,
 1999,
 2141,
 2582,
 2493,
 3875,
 881,
 2910,
 3923,
 2205,
 1966,
 1918,
 2241,
 1188,
 1672,
 1551,
 2617,
 2717,
 904,
 843,
 1510,
 451,
 3409,
 1013,
 761,
 834,
 1200,
 1456,
 514,
 885,
 1442,
 1539,
 1479,
 0,
 0,
 2826,
 0,
 0,
 3010,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3965,
 1175,
 1342,
 1688,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 826,
 0,
 0,
 290,
 1820,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4409,
 4918,
 0,
 0,
 0,
 0,
 2994,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,

In [27]:
standard2['total']=total_list
standard2.to_csv('standard3.csv',encoding='euc-kr')

In [28]:
standard4=pd.read_csv('standard_4.csv',encoding="CP949")

In [29]:
standard4.drop(['Unnamed: 0'],axis=1,inplace=True)
standard4.head()

,dong,gu,total
0,가락1동,송파구,3742
1,가락2동,송파구,3288
2,가락본동,송파구,1960
3,가리봉동,구로구,290
4,가산동,금천구,826


In [30]:
XY_dong=pd.read_csv("data/행정_법정동 중심좌표_v2.csv",encoding="CP949")

In [31]:
XY_dong.head()

,코드,시도,시군구,읍면동,하위,위도,경도,코드종류
0,4311374700,충청북도,청주시 흥덕구,가경동,NaN,36.620281,127.434219,H
1,4311311300,충청북도,청주시 흥덕구,가경동,NaN,36.622409,127.433548,B
2,4827054000,경상남도,밀양시,가곡동,NaN,35.476740,128.779272,H
3,4615010900,전라남도,순천시,가곡동,NaN,34.979840,127.483202,B
4,4827010800,경상남도,밀양시,가곡동,NaN,35.477144,128.767743,B


In [32]:
XY_dong2=XY_dong.drop(XY_dong[XY_dong['시도']!='서울특별시'].index)
#XY_dong3=XY_dong2.drop(XY_dong2[XY_dong['코드종류']!='H'].index)

In [33]:
XY_dong2.drop(['코드','시도','시군구','하위','코드종류'],axis=1,inplace=True)

In [34]:
XY_dong2.head()

,읍면동,위도,경도
61,가락1동,37.496537,127.105726
62,가락2동,37.498639,127.126488
64,가락동,37.496486,127.126006
65,가락본동,37.495585,127.121830
77,가리봉동,37.482704,126.888019


In [35]:
XY_dong2.rename(columns={"읍면동":"dong"},inplace=True)

In [36]:
XY_dong2=XY_dong2.reset_index()

In [37]:
X_list=[]
Y_list=[]
for i in range(len(standard4)):
    X_list.append(0)
    Y_list.append(0)
    for j in range(len(XY_dong2)):
        if standard4.dong[i]==XY_dong2.dong[j]:
            X_list[i]=XY_dong2.위도[j]
            Y_list[i]=XY_dong2.경도[j]
            break;

In [38]:
len(X_list)

423

In [39]:
standard5=standard4
standard5['X']=X_list
standard5['Y']=Y_list
standard5[standard5['X']==0].index

Int64Index([], dtype='int64')

In [4]:
# 중심 위도, 경도를 추가

In [40]:
standard5

,dong,gu,total,X,Y
0,가락1동,송파구,3742,37.496537,127.105726
1,가락2동,송파구,3288,37.498639,127.126488
2,가락본동,송파구,1960,37.495585,127.121830
3,가리봉동,구로구,290,37.482704,126.888019
4,가산동,금천구,826,37.476681,126.883767
...,...,...,...,...,...
418,효창동,용산구,911,37.542828,126.960951
419,후암동,용산구,1195,37.549901,126.980613
420,휘경1동,동대문구,915,37.574428,127.039712
421,휘경2동,동대문구,1631,37.574428,127.039712


In [41]:
icare=pd.read_csv('data/kium_center_v2.CSV',encoding = 'CP949')
icare.head()

,no,자치구,동,시설명,주소,위도,경도
0,1,강남구,수서동,강남1호점 우리동네키움센터(강남 다함께키움센터(태화)),서울특별시 강남구 광평로 185,37.484911,127.093399
1,2,강남구,역삼2동,강남2호점 우리동네키움센터(강남 다함께키움센터(역삼)),서울특별시 강남구 논현로64길 7,37.494121,127.040921
2,3,강남구,청담동,강남3호점 우리동네키움센터(강남 다함께키움센터(청담)),서울특별시 강남구 영동대로131길 26,37.520577,127.054106
3,4,강남구,세곡동,강남4호점 우리동네키움센터(미미위세곡키움센터),서울특별시 강남구 밤고개로 231,37.473740,127.107240
4,5,강동구,암사1동,강동1호점 우리동네키움센터(상상나루來키움센터),서울특별시 강동구 올림픽로98가길 28,37.550730,127.131430


In [42]:
icare.rename(columns={"자치구":"gu","동":"dong","시설명":"name","주소":"full_address",
                      "위도":"X","경도":"Y"},inplace=True)

In [43]:
icare.head()

,no,gu,dong,name,full_address,X,Y
0,1,강남구,수서동,강남1호점 우리동네키움센터(강남 다함께키움센터(태화)),서울특별시 강남구 광평로 185,37.484911,127.093399
1,2,강남구,역삼2동,강남2호점 우리동네키움센터(강남 다함께키움센터(역삼)),서울특별시 강남구 논현로64길 7,37.494121,127.040921
2,3,강남구,청담동,강남3호점 우리동네키움센터(강남 다함께키움센터(청담)),서울특별시 강남구 영동대로131길 26,37.520577,127.054106
3,4,강남구,세곡동,강남4호점 우리동네키움센터(미미위세곡키움센터),서울특별시 강남구 밤고개로 231,37.473740,127.107240
4,5,강동구,암사1동,강동1호점 우리동네키움센터(상상나루來키움센터),서울특별시 강동구 올림픽로98가길 28,37.550730,127.131430


In [44]:
icare.drop(['name','gu','full_address'],axis=1,inplace=True)

In [45]:
icare_grouped = icare['dong'].groupby(icare['dong']).count()
icare_grouped

dong
가락1동     1
가락본동     1
갈현2동     1
개봉3동     1
고척1동     1
        ..
청운효자동    1
청파동      1
하계2동     1
한강로동     1
한남동      1
Name: dong, Length: 90, dtype: int64

In [46]:
icare_grouped=icare_grouped.to_frame()
icare_grouped

,dong
dong,
가락1동,1
가락본동,1
갈현2동,1
개봉3동,1
고척1동,1
...,...
청운효자동,1
청파동,1
하계2동,1


In [47]:
icare_grouped.rename(columns={"dong":"icare_cnt"},inplace=True)

In [48]:
icare_grouped.head()

,icare_cnt
dong,
가락1동,1
가락본동,1
갈현2동,1
개봉3동,1
고척1동,1


In [49]:
icare_grouped.rename(columns={"dong":"icare_cnt"},inplace=True)

In [50]:
icare_grouped.index.name = 'dong'
icare_grouped=icare_grouped.reset_index()
icare_grouped.head()

,dong,icare_cnt
0,가락1동,1
1,가락본동,1
2,갈현2동,1
3,개봉3동,1
4,고척1동,1


In [51]:
standard6=pd.merge(standard5,icare_grouped,on='dong',how='left')

In [52]:
standard6=standard6.fillna(0)

In [5]:
# 동 별 우리동네키움센터 갯수 추가

In [53]:
standard6

,dong,gu,total,X,Y,icare_cnt
0,가락1동,송파구,3742,37.496537,127.105726,1.0
1,가락2동,송파구,3288,37.498639,127.126488,0.0
2,가락본동,송파구,1960,37.495585,127.121830,1.0
3,가리봉동,구로구,290,37.482704,126.888019,0.0
4,가산동,금천구,826,37.476681,126.883767,0.0
...,...,...,...,...,...,...
418,효창동,용산구,911,37.542828,126.960951,0.0
419,후암동,용산구,1195,37.549901,126.980613,0.0
420,휘경1동,동대문구,915,37.574428,127.039712,0.0
421,휘경2동,동대문구,1631,37.574428,127.039712,0.0


In [54]:
standard6.isnull().sum()

dong         0
gu           0
total        0
X            0
Y            0
icare_cnt    0
dtype: int64

In [55]:
center=pd.read_csv('data/지역아동센터3.csv',encoding="CP949")
center.head()

,소재구,센터명,도로명주소,지번주소,위도,경도,dong
0,동작구,푸른나무지역아동센터,서울특별시 동작구 흑석한강로 13,서울특별시 동작구 흑석동 335-3,37.505647,126.966572,흑석동
1,동작구,흑석희망지역아동센터,서울특별시 동작구 현충로 111,서울특별시 동작구 흑석동 10,37.507623,126.966050,흑석동
2,동작구,흑석지역아동센터,서울특별시 동작구 현충로 111,서울특별시 동작구 흑석동 10,37.507623,126.966050,흑석동
3,동작구,선한지역아동센터,서울특별시 동작구 흑석로13마길 1,서울특별시 동작구 흑석동 173-30,37.509464,126.960051,흑석동
4,동대문구,해피아트지역아동센터,서울특별시 동대문구 망우로14가길 28,서울특별시 동대문구 휘경동 294-152,37.587925,127.059544,휘경2동


In [56]:
center.columns

Index(['소재구', '센터명', '도로명주소', '지번주소', '위도', '경도', 'dong'], dtype='object')

In [57]:
 center.rename(columns={"소재구":"gu","Unnamed: 6":"dong","센터명":"name",
                    "위도":"X","경도":"Y"},inplace=True)

In [58]:
center_grouped = center['dong'].groupby(center['dong']).count()

In [59]:
center_grouped=center_grouped.to_frame()
center_grouped

,dong
dong,
가락2동,1
가리봉동,1
가산동,2
갈현1동,2
갈현2동,3
...,...
화곡6동,1
화곡본동,5
후암동,2


In [60]:
center_grouped.rename(columns={"dong":"center_cnt"},inplace=True)

In [61]:
center_grouped.index.name = 'dong'
center_grouped=center_grouped.reset_index()
center_grouped

,dong,center_cnt
0,가락2동,1
1,가리봉동,1
2,가산동,2
3,갈현1동,2
4,갈현2동,3
...,...,...
244,화곡6동,1
245,화곡본동,5
246,후암동,2
247,휘경2동,2


In [6]:
# 동 별 지역아동센터 갯수 추가. 지역아동센터와 서비스는 다르나 성격이 매우 유사함.

In [62]:
standard7=pd.merge(standard6, center_grouped, on="dong",how="left")
standard7.head()

,dong,gu,total,X,Y,icare_cnt,center_cnt
0,가락1동,송파구,3742,37.496537,127.105726,1.0,NaN
1,가락2동,송파구,3288,37.498639,127.126488,0.0,1.0
2,가락본동,송파구,1960,37.495585,127.121830,1.0,NaN
3,가리봉동,구로구,290,37.482704,126.888019,0.0,1.0
4,가산동,금천구,826,37.476681,126.883767,0.0,2.0


In [63]:
standard7=standard7.fillna(0)
standard7.head()

,dong,gu,total,X,Y,icare_cnt,center_cnt
0,가락1동,송파구,3742,37.496537,127.105726,1.0,0.0
1,가락2동,송파구,3288,37.498639,127.126488,0.0,1.0
2,가락본동,송파구,1960,37.495585,127.121830,1.0,0.0
3,가리봉동,구로구,290,37.482704,126.888019,0.0,1.0
4,가산동,금천구,826,37.476681,126.883767,0.0,2.0


In [64]:
def distance(df1,df2):
    LIST=[]
    for i in range(0,len(df1)):
        min_dis=10000
        for j in range(0,len(df2)):
            dis=haversine((df1['X'][i],df1['Y'][i]), (df2['X'][j],df2['Y'][j]), unit='nmi')
            if min_dis>dis:
                min_dis=dis
        LIST.append(min_dis)
    return LIST

In [65]:
center_distance=distance(standard7,center)
icare_distance=distance(standard7,icare)

In [66]:
standard7['center_distance(km)']=center_distance
standard7['icare_distance(km)']=icare_distance

In [7]:
# 동 별 갯수에 멈추지 않고, 가장 가까운 우리동네키움센터, 지역아동센터 거리(km)를 추가

In [67]:
standard7.head()

,dong,gu,total,X,Y,icare_cnt,center_cnt,center_distance(km),icare_distance(km)
0,가락1동,송파구,3742,37.496537,127.105726,1.0,0.0,0.433395,0.219154
1,가락2동,송파구,3288,37.498639,127.126488,0.0,1.0,0.147289,0.200885
2,가락본동,송파구,1960,37.495585,127.121830,1.0,0.0,0.198484,0.327061
3,가리봉동,구로구,290,37.482704,126.888019,0.0,1.0,0.156951,0.466515
4,가산동,금천구,826,37.476681,126.883767,0.0,2.0,0.478179,0.724643


In [68]:
gu_income=pd.read_csv("data/국민연금공단_자격 시구신고 평균소득월액_20200531.csv",encoding="CP949")

In [69]:
gu_income.head()

,기준년월,시군구,평균소득월액,Unnamed: 3
0,May-20,서울특별시,1450383,종로구
1,May-20,서울특별시,1451062,중구
2,May-20,서울특별시,1492947,용산구
3,May-20,서울특별시,1471625,성동구
4,May-20,서울특별시,1368980,광진구


In [70]:
gu_income=gu_income.drop(gu_income[gu_income['시군구']!='서울특별시'].index)
gu_income=gu_income.drop(gu_income[gu_income['기준년월']!='May-20'].index)

In [71]:
gu_income_grouped = gu_income['평균소득월액'].groupby(gu_income['Unnamed: 3']).sum()

In [72]:
gu_income_grouped

Unnamed: 3
강남구     1670051
강동구     1406722
강북구     1298784
강서구     1357676
관악구     1314745
광진구     1368980
구로구     1380680
금천구     1326957
노원구     1381690
도봉구     1342237
동대문구    1387911
동작구     1403742
마포구     1463518
서대문구    1397704
서초구     1654086
성동구     1471625
성북구     1386843
송파구     1492916
양천구     1427546
영등포구    1438335
용산구     1492947
은평구     1358067
종로구     1450383
중구      1451062
중랑구     1307629
Name: 평균소득월액, dtype: int64

In [73]:
gu_income_grouped=gu_income_grouped.to_frame()
gu_income.head()

,기준년월,시군구,평균소득월액,Unnamed: 3
0,May-20,서울특별시,1450383,종로구
1,May-20,서울특별시,1451062,중구
2,May-20,서울특별시,1492947,용산구
3,May-20,서울특별시,1471625,성동구
4,May-20,서울특별시,1368980,광진구


In [74]:
gu_income_grouped=gu_income_grouped.reset_index()
gu_income_grouped.rename(columns={"Unnamed: 3":"gu"},inplace=True)
gu_income_grouped.head()

,gu,평균소득월액
0,강남구,1670051
1,강동구,1406722
2,강북구,1298784
3,강서구,1357676
4,관악구,1314745


In [75]:
standard8=pd.merge(standard7, gu_income_grouped,on="gu",how='left')

In [8]:
# 소득 수준을 알기위해 건강보험 납입을 퇘로 평균소득월액 구 별 통계 추가

In [76]:
standard8.head()

,dong,gu,total,X,Y,icare_cnt,center_cnt,center_distance(km),icare_distance(km),평균소득월액
0,가락1동,송파구,3742,37.496537,127.105726,1.0,0.0,0.433395,0.219154,1492916
1,가락2동,송파구,3288,37.498639,127.126488,0.0,1.0,0.147289,0.200885,1492916
2,가락본동,송파구,1960,37.495585,127.121830,1.0,0.0,0.198484,0.327061,1492916
3,가리봉동,구로구,290,37.482704,126.888019,0.0,1.0,0.156951,0.466515,1380680
4,가산동,금천구,826,37.476681,126.883767,0.0,2.0,0.478179,0.724643,1326957


In [77]:
standard8.isnull().sum()

dong                   0
gu                     0
total                  0
X                      0
Y                      0
icare_cnt              0
center_cnt             0
center_distance(km)    0
icare_distance(km)     0
평균소득월액                 0
dtype: int64

In [78]:
school=pd.read_csv('data/Seoul_school_v4.csv',encoding="CP949")
school.head()

,행정구,행정동,시도교육청,지역교육청,지역,정보공시 학교코드,학교명,학교급코드,설립구분,학교특성,...,우편번호,학교도로명 우편번호,학교도로명 주소,학교도로명 상세주소,위도,경도,전화번호,팩스번호,홈페이지 주소,남녀공학 구분
0,강남구,개포2동,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 강남구,S010000738,개일초등학교,2,공립,NaN,...,135804.0,6323,서울특별시 강남구 개포로 401,", 개일초등학교 (개포동,개일초등학교)",37.486214,127.057741,02-571-8320,02-571-8325,http://gaeil.sen.es.kr,남녀공학
1,강남구,개포4동,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 강남구,S010000741,구룡초등학교,2,공립,NaN,...,135244.0,6305,서울특별시 강남구 개포로 263,"(개포동,구룡초등학교)",37.481197,127.051771,02-573-5017,02-575-4237,http://kuryong.sen.es.kr,남녀공학
2,강남구,일원2동,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 강남구,S010000747,대진초등학교,2,공립,NaN,...,135939.0,6334,서울특별시 강남구 개포로109길 74,", 서울대진초등학교 (개포동)",37.497012,127.078080,02-3412-0472,02-3412-0478,http://seouldaejin.sen.es.kr,남녀공학
3,강남구,개포2동,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 강남구,S010000771,양전초등학교,2,공립,NaN,...,135243.0,6328,서울특별시 강남구 개포로 513,", 서울양전초등학교",37.490283,127.069425,02-3411-8982,02-3411-8986,http://yangjeon.sen.es.kr,남녀공학
4,강남구,개포4동,서울특별시교육청,서울특별시강남서초교육지원청,서울특별시 강남구,S010000785,포이초등학교,2,공립,NaN,...,135961.0,6311,서울특별시 강남구 개포로22길 87,", 서울포이초등학교 (개포동,포이초등학교)",37.475916,127.052511,02-578-7334,02-571-3973,http://poi.sen.es.kr,남녀공학


In [79]:
school.rename(columns={"행정구":"gu","행정동":"dong","학교명":"name",
                      "위도":"X","경도":"Y"},inplace=True)

In [80]:
school.drop(['시도교육청', '지역교육청', '지역', '정보공시 학교코드', '학교급코드',
            '설립구분', '학교특성', '분교여부', '설립유형', '주야구분', '개교기념일',
            '설립일', '주소내역','상세주소내역', '우편번호', '학교도로명 우편번호',
            '학교도로명 주소', '학교도로명 상세주소','법정동코드',
            '전화번호', '팩스번호', '홈페이지 주소', '남녀공학 구분'],axis=1,inplace=True)

In [81]:
school.head()

,gu,dong,name,X,Y
0,강남구,개포2동,개일초등학교,37.486214,127.057741
1,강남구,개포4동,구룡초등학교,37.481197,127.051771
2,강남구,일원2동,대진초등학교,37.497012,127.078080
3,강남구,개포2동,양전초등학교,37.490283,127.069425
4,강남구,개포4동,포이초등학교,37.475916,127.052511


In [82]:
school_care=pd.read_csv('data/초등돌봄교실이용현황_서울.csv',encoding="CP949")

In [83]:
school_care.head()

,연번,시도,시군구,학교명,교실수,이용인원,소재지,비고,name
0,1,서울,종로구,교동초등학교,3,50,서울특별시 종로구 삼일대로 446 (경운동),경운동,교동초등학교
1,2,서울,종로구,독립문초등학교,4,80,서울특별시 종로구 통일로12길 23 (무악동),무악동,독립문초등학교
2,3,서울,종로구,매동초등학교,3,65,서울특별시 종로구 사직로9길 19 . 서울매동초등학교 (필운동),필운동,매동초등학교
3,4,서울,종로구,명신초등학교,3,60,서울특별시 종로구 낙산길 250 (창신동),창신동,명신초등학교
4,5,서울,종로구,세검정초등학교,5,111,서울특별시 종로구 세검정로9길 1 (신영동.서울세검정초등학교),신영동,세검정초등학교


In [84]:
school_care.rename(columns={"시군구":"gu","비고":"dong","이용인원":"방과후학교_이용가능인원"},inplace=True)

In [85]:
school_care.drop(["연번","시도","교실수","소재지",'학교명','gu','dong'],axis=1,inplace=True)

In [86]:
school_care.head()

,방과후학교_이용가능인원,name
0,50,교동초등학교
1,80,독립문초등학교
2,65,매동초등학교
3,60,명신초등학교
4,111,세검정초등학교


In [87]:
school.head()

,gu,dong,name,X,Y
0,강남구,개포2동,개일초등학교,37.486214,127.057741
1,강남구,개포4동,구룡초등학교,37.481197,127.051771
2,강남구,일원2동,대진초등학교,37.497012,127.078080
3,강남구,개포2동,양전초등학교,37.490283,127.069425
4,강남구,개포4동,포이초등학교,37.475916,127.052511


In [88]:
school

,gu,dong,name,X,Y
0,강남구,개포2동,개일초등학교,37.486214,127.057741
1,강남구,개포4동,구룡초등학교,37.481197,127.051771
2,강남구,일원2동,대진초등학교,37.497012,127.078080
3,강남구,개포2동,양전초등학교,37.490283,127.069425
4,강남구,개포4동,포이초등학교,37.475916,127.052511
...,...,...,...,...,...
597,중랑구,신내2동,신현초등학교,37.605815,127.093522
598,중랑구,신내2동,중화초등학교,37.602922,127.094065
599,중랑구,중화1동,새솔초등학교,37.617952,127.111328
600,중랑구,중화1동,묵동초등학교,37.603359,127.080827


In [89]:
school2=pd.merge(school,school_care,on='name',how='left')
school2=school2.drop_duplicates("name", keep="first")

In [90]:
school2

,gu,dong,name,X,Y,방과후학교_이용가능인원
0,강남구,개포2동,개일초등학교,37.486214,127.057741,44.0
1,강남구,개포4동,구룡초등학교,37.481197,127.051771,83.0
2,강남구,일원2동,대진초등학교,37.497012,127.078080,42.0
3,강남구,개포2동,양전초등학교,37.490283,127.069425,60.0
4,강남구,개포4동,포이초등학교,37.475916,127.052511,89.0
...,...,...,...,...,...,...
609,중랑구,신내2동,신현초등학교,37.605815,127.093522,57.0
610,중랑구,신내2동,중화초등학교,37.602922,127.094065,110.0
611,중랑구,중화1동,새솔초등학교,37.617952,127.111328,147.0
612,중랑구,중화1동,묵동초등학교,37.603359,127.080827,72.0


In [91]:
school2.isnull().sum()

gu               0
dong             0
name             0
X                0
Y                0
방과후학교_이용가능인원    62
dtype: int64

In [92]:
school2=school2.fillna(0)

In [93]:
school2_grouped=school2['방과후학교_이용가능인원'].groupby(school2['dong']).sum()
school2_grouped=school2_grouped.to_frame()

In [94]:
school2_grouped.head()

,방과후학교_이용가능인원
dong,
가락1동,66.0
가락2동,169.0
가락본동,115.0
가리봉동,77.0
가양1동,241.0


In [95]:
school2_grouped=school2_grouped.reset_index()
school2_grouped.head()

,dong,방과후학교_이용가능인원
0,가락1동,66.0
1,가락2동,169.0
2,가락본동,115.0
3,가리봉동,77.0
4,가양1동,241.0


In [96]:
standard9=pd.merge(standard8, school2_grouped,on="dong",how='left')

In [9]:
# 인근에 학교가 있으면 접근성이 좋을 것이며, 방과후학교를 운영하는 학교 갯수와 수용 가능 인원 데이터를 추가

In [97]:
standard9.head()

,dong,gu,total,X,Y,icare_cnt,center_cnt,center_distance(km),icare_distance(km),평균소득월액,방과후학교_이용가능인원
0,가락1동,송파구,3742,37.496537,127.105726,1.0,0.0,0.433395,0.219154,1492916,66.0
1,가락2동,송파구,3288,37.498639,127.126488,0.0,1.0,0.147289,0.200885,1492916,169.0
2,가락본동,송파구,1960,37.495585,127.121830,1.0,0.0,0.198484,0.327061,1492916,115.0
3,가리봉동,구로구,290,37.482704,126.888019,0.0,1.0,0.156951,0.466515,1380680,77.0
4,가산동,금천구,826,37.476681,126.883767,0.0,2.0,0.478179,0.724643,1326957,NaN


In [98]:
school2.head()

,gu,dong,name,X,Y,방과후학교_이용가능인원
0,강남구,개포2동,개일초등학교,37.486214,127.057741,44.0
1,강남구,개포4동,구룡초등학교,37.481197,127.051771,83.0
2,강남구,일원2동,대진초등학교,37.497012,127.078080,42.0
3,강남구,개포2동,양전초등학교,37.490283,127.069425,60.0
4,강남구,개포4동,포이초등학교,37.475916,127.052511,89.0


In [99]:
school2_grouped=school2['name'].groupby(school2['dong']).count()
school2_grouped=school2_grouped.to_frame()

In [100]:
school2_grouped.head()

,name
dong,
가락1동,2
가락2동,2
가락본동,2
가리봉동,1
가양1동,2


In [101]:
school2_grouped=school2_grouped.reset_index()
school2_grouped.head()

,dong,name
0,가락1동,2
1,가락2동,2
2,가락본동,2
3,가리봉동,1
4,가양1동,2


In [102]:
school2_grouped.isnull().sum()

dong    0
name    0
dtype: int64

In [103]:
school2_grouped=school2_grouped.rename({"name":"school_cnt"},axis=1)

In [104]:
standard10=pd.merge(standard9, school2_grouped,on="dong",how='left')

In [105]:
standard10=standard10.fillna(0)
standard10.head()

,dong,gu,total,X,Y,icare_cnt,center_cnt,center_distance(km),icare_distance(km),평균소득월액,방과후학교_이용가능인원,school_cnt
0,가락1동,송파구,3742,37.496537,127.105726,1.0,0.0,0.433395,0.219154,1492916,66.0,2.0
1,가락2동,송파구,3288,37.498639,127.126488,0.0,1.0,0.147289,0.200885,1492916,169.0,2.0
2,가락본동,송파구,1960,37.495585,127.121830,1.0,0.0,0.198484,0.327061,1492916,115.0,2.0
3,가리봉동,구로구,290,37.482704,126.888019,0.0,1.0,0.156951,0.466515,1380680,77.0,1.0
4,가산동,금천구,826,37.476681,126.883767,0.0,2.0,0.478179,0.724643,1326957,0.0,0.0


In [106]:
both_working=pd.read_csv("data/맞벌이.csv",encoding='CP949')

In [107]:
both_working.head()

,행정구,맞벌이 가정 비율
0,도봉구,61.58
1,광진구,56.76
2,성북구,55.21
3,구로구,54.91
4,관악구,54.89


In [108]:
both_working.rename(columns={"행정구":"gu","맞벌이_비율":"both_work_rate"},inplace=True)

In [109]:
both_working.head()

,gu,맞벌이 가정 비율
0,도봉구,61.58
1,광진구,56.76
2,성북구,55.21
3,구로구,54.91
4,관악구,54.89


In [10]:
# 맞벌이 가정 비율 데이터를 추가

In [110]:
standard11=pd.merge(standard10, both_working,on="gu",how='left')
standard11.head()

,dong,gu,total,X,Y,icare_cnt,center_cnt,center_distance(km),icare_distance(km),평균소득월액,방과후학교_이용가능인원,school_cnt,맞벌이 가정 비율
0,가락1동,송파구,3742,37.496537,127.105726,1.0,0.0,0.433395,0.219154,1492916,66.0,2.0,47.82
1,가락2동,송파구,3288,37.498639,127.126488,0.0,1.0,0.147289,0.200885,1492916,169.0,2.0,47.82
2,가락본동,송파구,1960,37.495585,127.121830,1.0,0.0,0.198484,0.327061,1492916,115.0,2.0,47.82
3,가리봉동,구로구,290,37.482704,126.888019,0.0,1.0,0.156951,0.466515,1380680,77.0,1.0,54.91
4,가산동,금천구,826,37.476681,126.883767,0.0,2.0,0.478179,0.724643,1326957,0.0,0.0,52.71


In [111]:
icare2=pd.read_csv('data/kium_center_v3.CSV',encoding="CP949")
icare2.head()

,no,자치구,동,시설명,주소,위도,경도,정원,상시이용,일시이용
0,1,강남구,수서동,강남1호점 우리동네키움센터(강남 다함께키움센터(태화)),서울특별시 강남구 광평로 185,37.484911,127.093399,45.0,38.0,3.0
1,2,강남구,역삼2동,강남2호점 우리동네키움센터(강남 다함께키움센터(역삼)),서울특별시 강남구 논현로64길 7,37.494121,127.040921,20.0,20.0,1.0
2,3,강남구,청담동,강남3호점 우리동네키움센터(강남 다함께키움센터(청담)),서울특별시 강남구 영동대로131길 26,37.520577,127.054106,20.0,22.0,6.0
3,4,강남구,세곡동,강남4호점 우리동네키움센터(미미위세곡키움센터),서울특별시 강남구 밤고개로 231,37.473740,127.107240,25.0,27.0,6.0
4,5,강동구,암사1동,강동1호점 우리동네키움센터(상상나루來키움센터),서울특별시 강동구 올림픽로98가길 28,37.550730,127.131430,20.0,20.0,2.0


In [112]:
icare2.columns

Index(['no', '자치구', '동', '시설명', '주소', '위도', '경도', '정원', '상시이용', '일시이용'], dtype='object')

In [113]:
icare2.drop(['no','시설명', '주소', '위도', '경도'],axis=1,inplace=True)

In [114]:
icare2.head()

,자치구,동,정원,상시이용,일시이용
0,강남구,수서동,45.0,38.0,3.0
1,강남구,역삼2동,20.0,20.0,1.0
2,강남구,청담동,20.0,22.0,6.0
3,강남구,세곡동,25.0,27.0,6.0
4,강동구,암사1동,20.0,20.0,2.0


In [115]:
icare2.isnull().sum()

자치구     0
동       0
정원      6
상시이용    6
일시이용    6
dtype: int64

In [116]:
icare2['정원']=icare2['정원'].fillna(1)
icare2=icare2.fillna(0)

In [117]:
icare2.head()

,자치구,동,정원,상시이용,일시이용
0,강남구,수서동,45.0,38.0,3.0
1,강남구,역삼2동,20.0,20.0,1.0
2,강남구,청담동,20.0,22.0,6.0
3,강남구,세곡동,25.0,27.0,6.0
4,강동구,암사1동,20.0,20.0,2.0


In [118]:
icare2.rename(columns={"자치구":"gu","동":"dong"},inplace=True)

In [119]:
icare2

,gu,dong,정원,상시이용,일시이용
0,강남구,수서동,45.0,38.0,3.0
1,강남구,역삼2동,20.0,20.0,1.0
2,강남구,청담동,20.0,22.0,6.0
3,강남구,세곡동,25.0,27.0,6.0
4,강동구,암사1동,20.0,20.0,2.0
...,...,...,...,...,...
101,중구,중림동,40.0,31.0,0.0
102,중구,중림동,40.0,38.0,0.0
103,중구,신당동,80.0,58.0,1.0
104,중구,신당5동,30.0,32.0,0.0


In [120]:
icare_grouped_total=icare2['정원'].groupby(icare2['gu']).sum()

In [121]:
icare_grouped_total

gu
강남구     110.0
강동구      20.0
강북구       2.0
강서구      45.0
광진구      70.0
구로구     228.0
노원구     600.0
도봉구      60.0
동대문구     50.0
동작구      80.0
마포구      50.0
서대문구     40.0
서초구      65.0
성동구     192.0
성북구      29.0
송파구     175.0
양천구      25.0
영등포구    135.0
용산구     165.0
은평구      80.0
종로구      40.0
중구      220.0
중랑구      22.0
Name: 정원, dtype: float64

In [122]:
icare_grouped_total.index.name = 'gu'
icare_grouped_total=icare_grouped_total.reset_index()
icare_grouped_total.head()

,gu,정원
0,강남구,110.0
1,강동구,20.0
2,강북구,2.0
3,강서구,45.0
4,광진구,70.0


In [12]:
# 우리동네키움센터에 문의 끝에 현재 이용 현황을 얻어 수용 가능 인원 / 현재 인원(상시, 수시) 등의 데이터를 추가

In [123]:
standard12=pd.merge(standard11, icare_grouped_total,on="gu",how="left")
standard12.head()

,dong,gu,total,X,Y,icare_cnt,center_cnt,center_distance(km),icare_distance(km),평균소득월액,방과후학교_이용가능인원,school_cnt,맞벌이 가정 비율,정원
0,가락1동,송파구,3742,37.496537,127.105726,1.0,0.0,0.433395,0.219154,1492916,66.0,2.0,47.82,175.0
1,가락2동,송파구,3288,37.498639,127.126488,0.0,1.0,0.147289,0.200885,1492916,169.0,2.0,47.82,175.0
2,가락본동,송파구,1960,37.495585,127.121830,1.0,0.0,0.198484,0.327061,1492916,115.0,2.0,47.82,175.0
3,가리봉동,구로구,290,37.482704,126.888019,0.0,1.0,0.156951,0.466515,1380680,77.0,1.0,54.91,228.0
4,가산동,금천구,826,37.476681,126.883767,0.0,2.0,0.478179,0.724643,1326957,0.0,0.0,52.71,NaN


In [124]:
icare_grouped_always=icare2['상시이용'].groupby(icare2['gu']).sum()
icare_grouped_always

gu
강남구     107.0
강동구      20.0
강북구       0.0
강서구      47.0
광진구      65.0
구로구     143.0
노원구     584.0
도봉구      66.0
동대문구     42.0
동작구      59.0
마포구      50.0
서대문구     34.0
서초구      61.0
성동구     154.0
성북구      32.0
송파구     126.0
양천구      33.0
영등포구    130.0
용산구      91.0
은평구      60.0
종로구      12.0
중구      180.0
중랑구      21.0
Name: 상시이용, dtype: float64

In [125]:
icare_grouped_always.index.name = 'gu'
icare_grouped_always=icare_grouped_always.reset_index()
icare_grouped_always.head()

,gu,상시이용
0,강남구,107.0
1,강동구,20.0
2,강북구,0.0
3,강서구,47.0
4,광진구,65.0


In [126]:
standard13=pd.merge(standard12, icare_grouped_always,on="gu",how="left")
standard13.head()

,dong,gu,total,X,Y,icare_cnt,center_cnt,center_distance(km),icare_distance(km),평균소득월액,방과후학교_이용가능인원,school_cnt,맞벌이 가정 비율,정원,상시이용
0,가락1동,송파구,3742,37.496537,127.105726,1.0,0.0,0.433395,0.219154,1492916,66.0,2.0,47.82,175.0,126.0
1,가락2동,송파구,3288,37.498639,127.126488,0.0,1.0,0.147289,0.200885,1492916,169.0,2.0,47.82,175.0,126.0
2,가락본동,송파구,1960,37.495585,127.121830,1.0,0.0,0.198484,0.327061,1492916,115.0,2.0,47.82,175.0,126.0
3,가리봉동,구로구,290,37.482704,126.888019,0.0,1.0,0.156951,0.466515,1380680,77.0,1.0,54.91,228.0,143.0
4,가산동,금천구,826,37.476681,126.883767,0.0,2.0,0.478179,0.724643,1326957,0.0,0.0,52.71,NaN,NaN


In [127]:
icare_grouped_sometimes=icare2['일시이용'].groupby(icare2['gu']).sum()
icare_grouped_sometimes

gu
강남구     16.0
강동구      2.0
강북구      0.0
강서구      1.0
광진구      2.0
구로구     16.0
노원구     20.0
도봉구     12.0
동대문구     5.0
동작구      7.0
마포구     11.0
서대문구     4.0
서초구      4.0
성동구     10.0
성북구      1.0
송파구     13.0
양천구      7.0
영등포구    41.0
용산구      8.0
은평구     19.0
종로구      5.0
중구       1.0
중랑구      0.0
Name: 일시이용, dtype: float64

In [128]:
icare_grouped_sometimes.index.name = 'gu'
icare_grouped_sometimes=icare_grouped_sometimes.reset_index()
icare_grouped_sometimes.head()

,gu,일시이용
0,강남구,16.0
1,강동구,2.0
2,강북구,0.0
3,강서구,1.0
4,광진구,2.0


In [129]:
standard14=pd.merge(standard13, icare_grouped_sometimes,on="gu",how="left")
standard14.head()

,dong,gu,total,X,Y,icare_cnt,center_cnt,center_distance(km),icare_distance(km),평균소득월액,방과후학교_이용가능인원,school_cnt,맞벌이 가정 비율,정원,상시이용,일시이용
0,가락1동,송파구,3742,37.496537,127.105726,1.0,0.0,0.433395,0.219154,1492916,66.0,2.0,47.82,175.0,126.0,13.0
1,가락2동,송파구,3288,37.498639,127.126488,0.0,1.0,0.147289,0.200885,1492916,169.0,2.0,47.82,175.0,126.0,13.0
2,가락본동,송파구,1960,37.495585,127.121830,1.0,0.0,0.198484,0.327061,1492916,115.0,2.0,47.82,175.0,126.0,13.0
3,가리봉동,구로구,290,37.482704,126.888019,0.0,1.0,0.156951,0.466515,1380680,77.0,1.0,54.91,228.0,143.0,16.0
4,가산동,금천구,826,37.476681,126.883767,0.0,2.0,0.478179,0.724643,1326957,0.0,0.0,52.71,NaN,NaN,NaN


In [130]:
standard14.columns

Index(['dong', 'gu', 'total', 'X', 'Y', 'icare_cnt', 'center_cnt',
       'center_distance(km)', 'icare_distance(km)', '평균소득월액', '방과후학교_이용가능인원',
       'school_cnt', '맞벌이 가정 비율', '정원', '상시이용', '일시이용'],
      dtype='object')

In [131]:
standard14.rename(columns={"정원":"icare_수용정원","상시이용":"icare_상시이용","일시이용":"icare_일시이용"},inplace=True)

In [132]:
standard14.head()

,dong,gu,total,X,Y,icare_cnt,center_cnt,center_distance(km),icare_distance(km),평균소득월액,방과후학교_이용가능인원,school_cnt,맞벌이 가정 비율,icare_수용정원,icare_상시이용,icare_일시이용
0,가락1동,송파구,3742,37.496537,127.105726,1.0,0.0,0.433395,0.219154,1492916,66.0,2.0,47.82,175.0,126.0,13.0
1,가락2동,송파구,3288,37.498639,127.126488,0.0,1.0,0.147289,0.200885,1492916,169.0,2.0,47.82,175.0,126.0,13.0
2,가락본동,송파구,1960,37.495585,127.121830,1.0,0.0,0.198484,0.327061,1492916,115.0,2.0,47.82,175.0,126.0,13.0
3,가리봉동,구로구,290,37.482704,126.888019,0.0,1.0,0.156951,0.466515,1380680,77.0,1.0,54.91,228.0,143.0,16.0
4,가산동,금천구,826,37.476681,126.883767,0.0,2.0,0.478179,0.724643,1326957,0.0,0.0,52.71,NaN,NaN,NaN


In [133]:
standard14['icare_수용정원']=standard14['icare_수용정원'].fillna(1)

In [134]:
standard14=standard14.fillna(1)

In [135]:
standard14.isnull().sum()

dong                   0
gu                     0
total                  0
X                      0
Y                      0
icare_cnt              0
center_cnt             0
center_distance(km)    0
icare_distance(km)     0
평균소득월액                 0
방과후학교_이용가능인원           0
school_cnt             0
맞벌이 가정 비율              0
icare_수용정원             0
icare_상시이용             0
icare_일시이용             0
dtype: int64

In [136]:
standard15=standard14

In [137]:
standard15

,dong,gu,total,X,Y,icare_cnt,center_cnt,center_distance(km),icare_distance(km),평균소득월액,방과후학교_이용가능인원,school_cnt,맞벌이 가정 비율,icare_수용정원,icare_상시이용,icare_일시이용
0,가락1동,송파구,3742,37.496537,127.105726,1.0,0.0,0.433395,0.219154,1492916,66.0,2.0,47.82,175.0,126.0,13.0
1,가락2동,송파구,3288,37.498639,127.126488,0.0,1.0,0.147289,0.200885,1492916,169.0,2.0,47.82,175.0,126.0,13.0
2,가락본동,송파구,1960,37.495585,127.121830,1.0,0.0,0.198484,0.327061,1492916,115.0,2.0,47.82,175.0,126.0,13.0
3,가리봉동,구로구,290,37.482704,126.888019,0.0,1.0,0.156951,0.466515,1380680,77.0,1.0,54.91,228.0,143.0,16.0
4,가산동,금천구,826,37.476681,126.883767,0.0,2.0,0.478179,0.724643,1326957,0.0,0.0,52.71,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,효창동,용산구,911,37.542828,126.960951,0.0,0.0,0.174172,0.081055,1492947,76.0,1.0,52.86,165.0,91.0,8.0
419,후암동,용산구,1195,37.549901,126.980613,0.0,2.0,0.292345,0.817499,1492947,129.0,2.0,52.86,165.0,91.0,8.0
420,휘경1동,동대문구,915,37.574428,127.039712,0.0,0.0,0.367682,0.366049,1387911,0.0,0.0,49.21,50.0,42.0,5.0
421,휘경2동,동대문구,1631,37.574428,127.039712,0.0,2.0,0.367682,0.366049,1387911,137.0,2.0,49.21,50.0,42.0,5.0


# 3. 이상치에 상대적으로 덜 민감한 RobustScaler로 값 스케일링

In [138]:
from sklearn.preprocessing import RobustScaler

def normalize_value(data,column):
    scaler = MinMaxScaler()
    data[column] = scaler.fit_transform(data[column].values.reshape(-1,1))
    return data

In [139]:
columns=standard15.columns
columns=columns[5:]
columns

Index(['icare_cnt', 'center_cnt', 'center_distance(km)', 'icare_distance(km)',
       '평균소득월액', '방과후학교_이용가능인원', 'school_cnt', '맞벌이 가정 비율', 'icare_수용정원',
       'icare_상시이용', 'icare_일시이용'],
      dtype='object')

In [140]:
for i in columns:
    standard15= normalize_value(standard15,i)

In [141]:
standard15.head()

,dong,gu,total,X,Y,icare_cnt,center_cnt,center_distance(km),icare_distance(km),평균소득월액,방과후학교_이용가능인원,school_cnt,맞벌이 가정 비율,icare_수용정원,icare_상시이용,icare_일시이용
0,가락1동,송파구,3742,37.496537,127.105726,0.25,0.000,0.339748,0.094891,0.522891,0.132530,0.285714,0.165049,0.290484,0.215753,0.317073
1,가락2동,송파구,3288,37.498639,127.126488,0.00,0.125,0.112785,0.086683,0.522891,0.339357,0.285714,0.165049,0.290484,0.215753,0.317073
2,가락본동,송파구,1960,37.495585,127.121830,0.25,0.000,0.153397,0.143369,0.522891,0.230924,0.285714,0.165049,0.290484,0.215753,0.317073
3,가리봉동,구로구,290,37.482704,126.888019,0.00,0.125,0.120449,0.206020,0.220585,0.154618,0.142857,0.595267,0.378965,0.244863,0.390244
4,가산동,금천구,826,37.476681,126.883767,0.00,0.250,0.375274,0.321987,0.075883,0.000000,0.000000,0.461772,0.000000,0.001712,0.024390


In [142]:
standard15.isnull().sum()

dong                   0
gu                     0
total                  0
X                      0
Y                      0
icare_cnt              0
center_cnt             0
center_distance(km)    0
icare_distance(km)     0
평균소득월액                 0
방과후학교_이용가능인원           0
school_cnt             0
맞벌이 가정 비율              0
icare_수용정원             0
icare_상시이용             0
icare_일시이용             0
dtype: int64

In [143]:
standard15.columns

Index(['dong', 'gu', 'total', 'X', 'Y', 'icare_cnt', 'center_cnt',
       'center_distance(km)', 'icare_distance(km)', '평균소득월액', '방과후학교_이용가능인원',
       'school_cnt', '맞벌이 가정 비율', 'icare_수용정원', 'icare_상시이용', 'icare_일시이용'],
      dtype='object')

# 4. 우선 입지 추천 구역 랭크

In [144]:
standard15['실질적필요수요인원_변수']=standard15['total']*standard15['맞벌이 가정 비율']
standard15['소득수준_변수']=2-standard15['평균소득월액']
standard15['수용가능인원지역요인_변수']=2*(standard15['center_distance(km)']+standard15['icare_distance(km)'])+(standard15['icare_상시이용']+standard15['icare_일시이용'])/(standard15['icare_수용정원']+1)-3*standard15['icare_cnt']-2.5*standard15['center_cnt']+standard15['school_cnt']*0.5
#standard15['수용가능인원지역요인_변수']=2*(standard15['center_distance(km)']+standard15['icare_distance(km)'])*(standard15['icare_상시이용']+standard15['icare_일시이용'])/(standard15['icare_수용정원']+1)/(2*standard15['icare_cnt']+1)/(2*standard15['center_cnt']+1)*standard15['school_cnt']*0.5

In [145]:
standard15.columns

Index(['dong', 'gu', 'total', 'X', 'Y', 'icare_cnt', 'center_cnt',
       'center_distance(km)', 'icare_distance(km)', '평균소득월액', '방과후학교_이용가능인원',
       'school_cnt', '맞벌이 가정 비율', 'icare_수용정원', 'icare_상시이용', 'icare_일시이용',
       '실질적필요수요인원_변수', '소득수준_변수', '수용가능인원지역요인_변수'],
      dtype='object')

In [146]:
standard16=standard15
standard16.drop(['icare_cnt', 'center_cnt','center_distance(km)', 'icare_distance(km)', '평균소득월액', '방과후학교_이용가능인원','school_cnt', '맞벌이 가정 비율', 'icare_수용정원', 'icare_상시이용', 'icare_일시이용'],axis=1,inplace=True)


In [147]:
standard16.head()

,dong,gu,total,X,Y,실질적필요수요인원_변수,소득수준_변수,수용가능인원지역요인_변수
0,가락1동,송파구,3742,37.496537,127.105726,617.611650,1.477109,0.675022
1,가락2동,송파구,3288,37.498639,127.126488,542.679612,1.477109,0.642182
2,가락본동,송파구,1960,37.495585,127.121830,323.495146,1.477109,0.399278
3,가리봉동,구로구,290,37.482704,126.888019,172.627427,1.779415,0.872435
4,가산동,금천구,826,37.476681,126.883767,381.423544,1.924117,0.795624


In [148]:
columns=standard16.columns
columns=columns[5:]
columns

Index(['실질적필요수요인원_변수', '소득수준_변수', '수용가능인원지역요인_변수'], dtype='object')

In [149]:
for i in columns:
    standard16= normalize_value(standard16,i)

In [150]:
standard17=standard16.drop(['total'],axis=1)

In [151]:
standard17['합산']=standard17['실질적필요수요인원_변수']*0.6+standard17['소득수준_변수']*0.3+1.3*standard17['수용가능인원지역요인_변수']

In [188]:
standard18.rename(columns={"수용가능인원지역요인_변수":"수요인원_변수","실질적필요수요인원_변수":"지역변수"},inplace=True)

In [196]:
standard18=standard18.sort_values(by=['합산'],axis=0,ascending=False)

In [197]:
standard18.head(30)

,dong,gu,X,Y,지역변수,소득수준_변수,수요인원_변수,합산
28,광장동,광진구,37.547835,127.104695,1.000000,0.810929,0.846176,1.943307
368,진관동,은평구,37.639681,126.937804,0.825874,0.840322,0.734849,1.702924
45,길음1동,성북구,37.589440,127.016817,0.999138,0.762815,0.584855,1.588639
300,우장산동,강서구,37.548198,126.841792,0.689491,0.841376,0.694419,1.568852
185,상일동,강동구,37.550583,127.168797,0.451528,0.709271,0.827467,1.559406
53,남현동,관악구,37.464115,126.977754,0.224698,0.957009,0.860858,1.541037
389,평창동,종로구,37.613029,126.974485,0.232182,0.591671,0.861690,1.437007
240,신도림동,구로구,37.509951,126.882174,0.827266,0.779415,0.543165,1.436299
162,삼각산동,강북구,37.616702,127.017511,0.552133,1.000000,0.608365,1.422155
224,시흥2동,금천구,37.456979,126.895656,0.293871,0.924117,0.716833,1.385440


In [154]:
standard13[standard13['dong']=="목5동"]

,dong,gu,total,X,Y,icare_cnt,center_cnt,center_distance(km),icare_distance(km),평균소득월액,방과후학교_이용가능인원,school_cnt,맞벌이 가정 비율,정원,상시이용
115,목5동,양천구,4908,37.535136,126.87854,0.0,0.0,0.503299,0.911172,1427546,237.0,4.0,48.92,25.0,33.0


In [155]:
standard13[standard13['dong']=="광장동"]

,dong,gu,total,X,Y,icare_cnt,center_cnt,center_distance(km),icare_distance(km),평균소득월액,방과후학교_이용가능인원,school_cnt,맞벌이 가정 비율,정원,상시이용
28,광장동,광진구,4484,37.547835,127.104695,0.0,0.0,0.760543,1.27998,1368980,310.0,3.0,56.76,70.0,65.0


In [156]:
standard13[standard13['dong']=="길음1동"]

,dong,gu,total,X,Y,icare_cnt,center_cnt,center_distance(km),icare_distance(km),평균소득월액,방과후학교_이용가능인원,school_cnt,맞벌이 가정 비율,정원,상시이용
45,길음1동,성북구,5167,37.58944,127.016817,0.0,1.0,0.246292,1.016409,1386843,290.0,3.0,55.21,29.0,32.0


# 5. folium, Json을 활용한 매핑 및 지도 시각화(데이터 시각화 다른 자료)

In [13]:
# 동 별 타게팅 인구수

In [157]:
#geo_path = 'data/02. skorea_municipalities_geo_simple.json'
geo_path = 'data/seoul_submunicipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))


map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                 tiles='Stamen Toner')
map.choropleth(geo_data = geo_str,
               data = standard6,
               columns = ['dong','total'],
               fill_color = 'YlGnBu', #puRd, YlGnBu
               key_on = 'feature.properties.name')
map

In [14]:
# 동 별 아이케어센터 수

In [158]:
import json
#geo_path = 'data/02. skorea_municipalities_geo_simple.json'
geo_path = 'data/seoul_submunicipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))


map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                 tiles='Stamen Toner')
map.choropleth(geo_data = geo_str,
               data = standard6,
               columns = ['dong','icare_cnt'],
               fill_color = 'YlGnBu', #puRd, YlGnBu
               key_on = 'feature.properties.name')
map

In [15]:
# 동 별 지역아동센터 수

In [159]:
import json
#geo_path = 'data/02. skorea_municipalities_geo_simple.json'
geo_path = 'data/seoul_submunicipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))


map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                 tiles='Stamen Toner')
map.choropleth(geo_data = geo_str,
               data = standard7,
               columns = ['dong','center_cnt'],
               fill_color = 'PuRd', #PuRd, YlGnBu
               key_on = 'feature.properties.name')
map

In [160]:
standard17

,dong,gu,X,Y,실질적필요수요인원_변수,소득수준_변수,수용가능인원지역요인_변수,합산
0,가락1동,송파구,37.496537,127.105726,0.194674,0.477109,0.477055,0.880109
1,가락2동,송파구,37.498639,127.126488,0.171055,0.477109,0.471067,0.858153
2,가락본동,송파구,37.495585,127.121830,0.101967,0.477109,0.426775,0.759121
3,가리봉동,구로구,37.482704,126.888019,0.054413,0.779415,0.513052,0.933440
4,가산동,금천구,37.476681,126.883767,0.120227,0.924117,0.499046,0.998131
...,...,...,...,...,...,...,...,...
418,효창동,용산구,37.542828,126.960951,0.135212,0.477026,0.478119,0.845790
419,후암동,용산구,37.549901,126.980613,0.177364,0.477026,0.532026,0.941159
420,휘경1동,동대문구,37.574428,127.039712,0.071928,0.759938,0.550211,0.986412
421,휘경2동,동대문구,37.574428,127.039712,0.128213,0.759938,0.462295,0.905892


In [16]:
# 동 별 입지 추천 랭크

In [198]:
import json
#geo_path = 'data/02. skorea_municipalities_geo_simple.json'
geo_path = 'data/seoul_submunicipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))


map = folium.Map(location=[37.5502, 126.982], zoom_start=10.5,
                 tiles='Stamen Toner')
map.choropleth(geo_data = geo_str,
               data = standard17,
               columns = ['dong','합산'],
               fill_color = 'YlGnBu', #PuRd, YlGnBu
               key_on = 'feature.properties.name')
map

In [17]:
# 랭크 도출 중 인원 관련 고려사항 랭크

In [162]:
import json
#geo_path = 'data/02. skorea_municipalities_geo_simple.json'
geo_path = 'data/seoul_submunicipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))


map = folium.Map(location=[37.5502, 126.982], zoom_start=10.5,
                 tiles='Stamen Toner')
map.choropleth(geo_data = geo_str,
               data = standard17,
               columns = ['dong','실질적필요수요인원_변수'],
               fill_color = 'PuRd', #PuRd, YlGnBu
               key_on = 'feature.properties.name')
map

In [18]:
# 랭크 도출 중 소득 관련 고려사항 랭크

In [164]:
import json
#geo_path = 'data/02. skorea_municipalities_geo_simple.json'
geo_path = 'data/seoul_submunicipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))


map = folium.Map(location=[37.5502, 126.982], zoom_start=10.5,
                 tiles='Stamen Toner')
map.choropleth(geo_data = geo_str,
               data = standard12,
               columns = ['dong','평균소득월액'],
               fill_color = 'PuRd', #PuRd, YlGnBu
               key_on = 'feature.properties.name')
map

In [165]:
gu_income_grouped

,gu,평균소득월액
0,강남구,1670051
1,강동구,1406722
2,강북구,1298784
3,강서구,1357676
4,관악구,1314745
5,광진구,1368980
6,구로구,1380680
7,금천구,1326957
8,노원구,1381690
9,도봉구,1342237


In [166]:
import json
geo_path = 'data/02. skorea_municipalities_geo_simple.json'
#geo_path = 'data/seoul_submunicipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))


map = folium.Map(location=[37.5502, 126.982], zoom_start=11,
                 tiles='Stamen Toner')
map.choropleth(geo_data = geo_str,
               data = gu_income_grouped,
               columns = ['gu','평균소득월액'],
               fill_color = 'YlGnBu', #puRd, YlGnBu
               key_on = 'feature.id')
map

In [167]:
standard17.columns

Index(['dong', 'gu', 'X', 'Y', '실질적필요수요인원_변수', '소득수준_변수', '수용가능인원지역요인_변수',
       '합산'],
      dtype='object')

In [19]:
# 랭크 도출 중 시설을 고려한 변수 랭크

In [168]:
import json
#geo_path = 'data/02. skorea_municipalities_geo_simple.json'
geo_path = 'data/seoul_submunicipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))


map = folium.Map(location=[37.5502, 126.982], zoom_start=10.5,
                 tiles='Stamen Toner')
map.choropleth(geo_data = geo_str,
               data = standard17,
               columns = ['dong','실질적필요수요인원_변수'],
               fill_color = 'YlGnBu', #PuRd, YlGnBu
               key_on = 'feature.properties.name')
map

In [169]:
import json
#geo_path = 'data/02. skorea_municipalities_geo_simple.json'
geo_path = 'data/seoul_submunicipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))


map = folium.Map(location=[37.5502, 126.982], zoom_start=10.5,
                 tiles='Stamen Toner')
map.choropleth(geo_data = geo_str,
               data = standard17,
               columns = ['dong','수용가능인원지역요인_변수'],
               fill_color = 'PuRd', #PuRd, YlGnBu
               key_on = 'feature.properties.name')
map

In [170]:
standard12.columns

Index(['dong', 'gu', 'total', 'X', 'Y', 'icare_cnt', 'center_cnt',
       'center_distance(km)', 'icare_distance(km)', '평균소득월액', '방과후학교_이용가능인원',
       'school_cnt', '맞벌이 가정 비율', '정원'],
      dtype='object')

In [20]:
# 구별 맞벌이 가정 비율 

In [171]:
import json
#geo_path = 'data/02. skorea_municipalities_geo_simple.json'
geo_path = 'data/seoul_submunicipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))


map = folium.Map(location=[37.5502, 126.982], zoom_start=10.5,
                 tiles='Stamen Toner')
map.choropleth(geo_data = geo_str,
               data = standard12,
               columns = ['dong','맞벌이 가정 비율'],
               fill_color = 'PuRd', #PuRd, YlGnBu
               key_on = 'feature.properties.name')
map

In [172]:
standard12.columns

Index(['dong', 'gu', 'total', 'X', 'Y', 'icare_cnt', 'center_cnt',
       'center_distance(km)', 'icare_distance(km)', '평균소득월액', '방과후학교_이용가능인원',
       'school_cnt', '맞벌이 가정 비율', '정원'],
      dtype='object')

In [21]:
# 동 별 학교 갯수

In [173]:
import json
#geo_path = 'data/02. skorea_municipalities_geo_simple.json'
geo_path = 'data/seoul_submunicipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))


map = folium.Map(location=[37.5502, 126.982], zoom_start=10.5,
                 tiles='Stamen Toner')
map.choropleth(geo_data = geo_str,
               data = standard12,
               columns = ['dong','school_cnt'],
               fill_color = 'YlGnBu', #PuRd, YlGnBu
               key_on = 'feature.properties.name')
map